In [18]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
import re
import pandas as pd
from sklearn.svm import LinearSVC
import pickle
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from gensim.sklearn_api import D2VTransformer

import nltk
import string

In [19]:
def clean_name(name):
    name = re.sub(r'[^\w]', ' ', name)
    name = re.sub(r'[,:._\-\[\]\d]', ' ', name)
    
    s = ''
    for i in tokenize(name.lower().strip()):
        s += ' ' + i
    return s#name.lower().strip()

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 3:
                continue
            tokens.append(word.lower())
    return tokens

def tokenize(text):
    stem = nltk.stem.SnowballStemmer('english')
    text = text.lower()
    
    for token in nltk.word_tokenize(text):
        if token in string.punctuation:
            continue
        yield stem.stem(token)

def find_outlets(clf, X_train):
    pred = clf.predict(X_train)
    d = train.copy()
    d['pred'] = pred
    
    return d[d.category_id != d.pred].groupby('category_id').apply(len).sort_values(), d

def write_submit(freq, clf):
    pickle.dump(freq, open('submit_names/tfidf', 'wb'))
    pickle.dump(clf, open('submit_names/clf_task1', 'wb'))
    !cd submit_names;zip submit_names.zip *  
    
def read_and_clean_data():
    train = pd.read_parquet('data_fusion_train.parquet')
    train = train[train.category_id != -1].drop_duplicates('item_name')
    train['item_name'] = train['item_name'].apply(clean_name)

    train.replace('', np.nan, inplace=True)
    train.dropna(subset = ['item_name'], inplace = True)

    return train.sample(frac=1)   

def train_model(train):
    X_train = freq.fit_transform(train.item_name)

    clf = LinearSVC()
    print(cross_val_score(clf, X_train, train.category_id, cv=3, scoring='f1_weighted'))
    _ = clf.fit(X_train, train.category_id)
    return clf, X_train

In [20]:
train = read_and_clean_data()
drop_outlets = True

In [21]:
freq = TfidfVectorizer(ngram_range=(1,5), analyzer='char_wb')
clf, X_train = train_model(train)

[0.82944562 0.82771735 0.83016789]


In [22]:
if drop_outlets == True:
    outlets, train_no_outlets = find_outlets(clf, X_train)
    train_no_outlets = train_no_outlets[train_no_outlets.category_id == train_no_outlets.pred]
    train_no_outlets.sample(frac=1) 
    clf, X_train = train_model(train_no_outlets)
    
write_submit(freq, clf)

[0.8550724  0.85509868 0.8575399 ]
updating: script.py (deflated 58%)
  adding: clf_task1 (deflated 80%)
  adding: tfidf (deflated 64%)


In [ ]:
#103 - походы
#41 - ход за телом
#42 - уход за телом
#46 - парфюмерия
#138 - дл/ дачи, пикноков
#108 - покраска
#167 - рукоделие
#163 - рукоделие
#31 - рукоделие
#27 - освещение
#106 - для дома
#54 - верхняя уличная одежда
#56 - одежда
#58 - платье женское
#150 - все для собак. Очень плохая группа, содержит еду, одежду...
#19 - канцелярия
#164 - канцелярия
#26 - канцелярия
#24 - книги. канцелярия
#20 - чтиво
#203 - упаковка, бумага
#56 - для автомобиля
#13 - для автомобиля
#111 - туалет, ванна
#90 - развлечения
#96 - для спорта
#55 - джинсы
#39 - зрение
#35 - благовония
#101 - еда для фитнеса
